In [1]:
import pandas as pd
import numpy as np
import cv2
import skimage
from skimage.transform import resize
from skimage.io import imread, imshow
import matplotlib.pyplot as plt

import tensorflow
from tensorflow import keras
from tensorflow.keras.models import load_model

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import tree
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error

In [2]:
train_df = pd.read_csv("BoneAge.csv")

train_df.head()

,id,boneage,male
0,1386,30,False
1,1392,162,True
2,1397,18,False
3,1401,132,False
4,1410,57,True


In [3]:
train_df = train_df.drop("male", axis=1)

train_df.head()

,id,boneage
0,1386,30
1,1392,162
2,1397,18
3,1401,132
4,1410,57


In [4]:
train_df["file_paths"] = train_df["id"].apply(lambda x: "./data/validation/" + str(x) + ".png")

train_df.head()

,id,boneage,file_paths
0,1386,30,./data/validation/1386.png
1,1392,162,./data/validation/1392.png
2,1397,18,./data/validation/1397.png
3,1401,132,./data/validation/1401.png
4,1410,57,./data/validation/1410.png


In [5]:
def find_valid_files(file_path):
    try:
        imread(file_path)
        return 1
    except FileNotFoundError:
        return 0
    
train_df["valid"] = train_df["file_paths"].apply(lambda x: find_valid_files(x))
train_df = train_df[train_df["valid"] == 1]
train_df = train_df.drop("valid", axis=1)

train_df.head()

,id,boneage,file_paths
800,9727,120,./data/validation/9727.png
801,9731,84,./data/validation/9731.png
802,9743,113,./data/validation/9743.png
803,9748,144,./data/validation/9748.png
804,9755,106,./data/validation/9755.png


In [ ]:
MODEL_PATH = "FINAL_UNET_MODEL.h5"
model = load_model(MODEL_PATH)

In [ ]:
def apply_model(file_path):
    image = imread(file_path)
    image = resize(image, (256, 256, 1), mode="constant", preserve_range=True)
    result = model.predict( np.expand_dims(image, 0) )[0]
    result = np.where(result >= np.mean(result), 1, 0)
    
    return result

train_df["masks"] = train_df["file_paths"].apply(lambda x: apply_model(x))
train_df.head()

In [ ]:
segm_img = train_df.iloc(0)[8]["masks"]
imshow(segm_img, cmap=plt.cm.gray)

In [ ]:
train_df["flattened_masks"] = train_df["masks"].apply(lambda x: x.flatten())
train_df.head()

In [ ]:
X_train = np.array(train_df["flattened_masks"].values.tolist())
y_train = np.array(train_df["boneage"])

print(X_train.shape)
print(y_train.shape)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, 
                                                    test_size = 0.2, random_state = 0)

In [ ]:
regressor = SVR(kernel = 'linear')
regressor.fit(X_train, y_train)

preds = regressor.predict(X_test)
print(np.max((preds - y_test) / 12), "years")
print(np.min((preds - y_test) / 12), "years")

error = mean_absolute_error(y_test, preds)
print(error, " months")
print(error / 12, " years")

In [ ]:
param = {'kernel' : ('linear', 'rbf', 'sigmoid'),'C' : [1,5,10],'degree' : [3,8],'coef0' : [0.01,10,0.5],'gamma' : ('auto','scale')},
param_search = GridSearchCV(SVR(), param, scoring='neg_mean_absolute_error', verbose = 3)

param_search.fit(X_train, y_train)

In [ ]:
param_search.best_params_

In [ ]:
opt_regressor = SVR(C=10, coef0=0.01, degree=3, gamma="scale", kernel="rbf")
opt_regressor.fit(X_train, y_train)

preds = opt_regressor.predict(X_test)
print(np.max((preds - y_test) / 12), "years")
print(np.min((preds - y_test) / 12), "years")

error = mean_absolute_error(y_test, preds)
print(error, " months")
print(error / 12, " years")

In [ ]:
clf = tree.DecisionTreeClassifier()
clf.fit(X_train, y_train)

preds = clf.predict(X_test)
print(np.max((preds - y_test) / 12), "years")
print(np.min((preds - y_test) / 12), "years")

error = mean_absolute_error(y_test, preds)
print(error, " months")
print(error / 12, " years")

In [ ]:
clf = RandomForestRegressor(verbose = 1)
clf.fit(X_train, y_train)

preds = clf.predict(X_test)
print(np.max((preds - y_test) / 12), "years")
print(np.min((preds - y_test) / 12), "years")

error = mean_absolute_error(y_test, preds)
print(error, " months")
print(error / 12, " years")

In [ ]:
nn_model = keras.Sequential([
      tensorflow.keras.layers.Input((65536,)),
      tensorflow.keras.layers.Dense(128, activation='relu'),
      tensorflow.keras.layers.Dense(64, activation='relu'),
      tensorflow.keras.layers.Dense(64, activation='relu'),
      tensorflow.keras.layers.Dense(64, activation='relu'),
      tensorflow.keras.layers.Dense(1)
  ])

nn_model.compile(loss='mean_absolute_error', optimizer="adam")
nn_model.summary()

In [ ]:
nn_model.fit(X_train, y_train, epochs=50, validation_split=0.25, 
             callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss', patience=7)])

In [ ]:
preds = nn_model.predict(X_test)
print(np.max((preds - y_test) / 12), "years")
print(np.min((preds - y_test) / 12), "years")

error = mean_absolute_error(y_test, preds)
print(error, " months")
print(error / 12, " years")